<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema3_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tema 3.4. Construcción automática de modelos ARIMA

## *Método artesanal*: a través de bucles:

Encontrar el mejor modelo de manera manual en muchas ocasiones no es operativo, sobre todo cuando tenemos que repetir la tarea con muchas series o con cierta frecuencia. Por ello, es aconsejable automatizar el proceso. Una opción es a través de bucles, probando todas las combinaciones posibles de (p,d,q)(P,D,Q) y quedándonos con la que arroje un menor AIC. Veamos cómo sería.

In [ ]:
# Instalamos librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams["figure.figsize"] = [10,4]  # Default figure size

In [ ]:
# Importamos los datos usados en el notebook 3.5
medic = pd.read_csv("https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/medicamentos_ventas.csv",index_col=0,parse_dates=True)

# Indicamos la frecuencia de los datos (para evitar luego los mensajes warnings en el output)
medic.index.freq = 'MS'

# Creamos train y test dataset
train=medic[:-24]
test=medic[-24:]

In [ ]:
# Definimos la función
from statsmodels.tsa.statespace.sarimax import SARIMAX

def manual_arima(data, max_p=3, max_d=2, max_q=3,  max_P=3,  max_D=2, max_Q=3, M=12):

    n=0
    best_aic = float('inf')
    best_params = None

    # Test all parameter combinations with nested loops
    for p in range(max_p + 1):
        for d in range(max_d + 1):
            for q in range(max_q + 1):
                for P in range(max_P + 1):
                    for D in range(max_D+1):
                        for Q in range(max_Q + 1):
                            model = SARIMAX(
                                data,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, M)
                                )
                            aic = model.fit().aic

                            n+=1
                            print(f"Model #{n}")
                            print(f"({p},{d},{q})x({P},{D},{Q}) - AIC: {aic}")

                            if aic < best_aic:
                                best_aic = aic
                                best_params = (p, d, q, P, D, Q)

    return best_params, best_aic

In [ ]:
train.index.freq = 'MS' # to avoid
best_params, best_aic=manual_arima(train, max_p=2, max_d=1, max_q=2,  max_P=1,  max_D=1, max_Q=1)

In [ ]:
print(best_params)
print(best_aic)

## ```statsforecast```

El método artesanal o manual es muy lento. Afortunadamente, hay alternativas. Una librería reciente y potente para construir modelos automáticamente es ```statsforecast```:  
https://nixtlaverse.nixtla.io/statsforecast/index.html

Ofrece una colección de modelos populares de predicción de series temporales univariantes (incluido el ARIMA) optimizados para un alto rendimiento y escalabilidad.

Leer esta guía [QUICK START](https://nixtlaverse.nixtla.io/statsforecast/docs/getting-started/getting_started_short.html).  

Como se indica en el enlace, el *input* de ```StatsForecast``` siempre es un dataframe en formato largo con tres columnas: ```unique_id```, ```ds``` e ```y```.

* La columna ```unique_id``` (*string*, entero o categoría) representa un identificador para la serie.

* La columna ```ds``` (*datestamp*) debe tener un formato de fechas/periodos compatible con Pandas, idealmente AAAA-MM-DD.

* La columna ```y``` (numérica) representa la medición que se desea predecir.

### Instalación en Google Colab

Dado que la librería no viene pre-instalada en Google Colab, es necesario instalarla cada vez que abrimos una sesión en Google Colab. Esto toma unos segundos.

In [ ]:
pip install statsforecast

### Importar librerías y procesar datos

A continuación importamos la librería recien instalada, junto a la función AutoARIMA de manera separada, así como otras librerías que vamos a usar.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams["figure.figsize"] = [10,4]  # Default figure size
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, ARIMA

Es necesario procesar la dataframe ligeramente para ponerla en el formato aceptado por *StatsForecast*:

In [ ]:
medic = pd.read_csv("https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/medicamentos_ventas.csv",index_col=0,parse_dates=True)
medic["unique_id"]="ventas_medic"
medic["ds"]=medic.index
medic['y']=medic['value']
medic.drop(columns=['value'],inplace=True)

Dividimos la serie en train y test dataset:

In [ ]:
train=medic[:-24]
test=medic[-24:]

## Especificación y estimación del modelo AutoARIMA

Se especifica y se estima el modelo (tarda unos segundos porque estima muchos modelos hasta encontrar el óptimo):

In [ ]:
modelo = StatsForecast(models=[AutoARIMA(season_length=12,
                                         max_p=2, max_d=1, max_q=2, max_P=1, max_D=1, max_Q=1,    # Para replicar los valores máximos establecidos en el método manual
                                          )],freq='MS')
modelo.fit(train)

El *output* del modelo se puede observar como sigue:

In [ ]:
print ("Modelo: (p,q,P,Q,M,d,D): ",modelo.fitted_[0,0].model_['arma'])
print ("Observations",modelo.fitted_[0,0].model_['nobs'])
print ("AIC: ",modelo.fitted_[0,0].model_['aic'])
print ("Coeficientes: ")
for x in modelo.fitted_[0,0].model_['coef']:
  print("  ", x, ":", modelo.fitted_[0,0].model_['coef'][x])

Para predecir existen dos métodos:
* `predict()` se ejecuta después de `fit()` y permite obtener los parámetros de cada modelo. Toma dos argumentos principales: **h** (número de periodos de la predicción) y **level** (una lista con los intervalos de confianza a calcular).
* `forecast()` se ejecuta directamente después de especificar el modelo (sin necesidad de estimarlo, porque ya lo hace internamente), pero no almacena ningún parámetro del modelo. Además de los dos argumentos anteriormente mencionados, un argumento adicional es: **df** (dataframe).

Vamos a predecir para el periodo de la test dataset y posteriormente comprobar cómo de exacto es nuestro modelo para predecir:

In [ ]:
forecast_df=modelo.predict(h=len(test), level=[95])
forecast_df.head() # es una dataframe con la predicción y el IC

Dibujamos la predicción:

In [ ]:
# Primero creo un index para forecast_df
forecast_df.set_index(forecast_df.ds, drop=False, inplace=True )

# Gráfico
fig,ax=plt.subplots(figsize=(10,7))
ax.plot(train[-60:]['y'], label="Training period", color="b") #con [-60:] evito representar la serie completa
ax.plot(test['y'], label="Actual values", color="k")
ax.plot(forecast_df['AutoARIMA'], label="Predicted", color='r')
ax.fill_between(forecast_df.index,
                 forecast_df['AutoARIMA-lo-95'],
                 forecast_df['AutoARIMA-hi-95'],
                 color='k', alpha=.15)
ax.legend()
ax.set_title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

Comprobamos numéricamente la exactitud de la predicción:

In [ ]:
def forecast_accuracy(forecast, actual):
    rmse = np.sqrt(np.mean((forecast-actual)**2))
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    print({'rmse':rmse,'mape':mape,'corr':corr})

forecast_accuracy(forecast_df['AutoARIMA'], test.y)

Predecimos también hacia el futuro:

In [ ]:
# Especificamos un modelo con los parámetros hallados anteriormente => Usamos el método ARIMA
# Modelo: (p,q,P,Q,M,d,D):  (2, 1, 0, 1, 12, 1, 1) equivale a order=(2, 1, 1) y seasonal_order=(0,1,1)

modelo_fut = StatsForecast(models=[ARIMA(order=(2, 1, 1), seasonal_order=(0,1,1),
                                     include_mean=False, season_length=12)],freq='MS')

forecast_df2=modelo_fut.forecast(df=medic, h=24, level=[95])
forecast_df2.set_index(forecast_df2.ds, drop=False, inplace=True )

# Gráfico
fig,ax=plt.subplots(figsize=(12,8))
ax.plot(medic.loc["2004":,'y'], label="Ventas de medicamentos", color="b") #con [-60:] evito representar la serie completa
ax.plot(forecast_df2['ARIMA'], label="Predicción", color='r')
ax.fill_between(forecast_df2.index,
                 forecast_df2['ARIMA-lo-95'],
                 forecast_df2['ARIMA-hi-95'],
                 color='k', alpha=.15)
ax.legend()
ax.set_title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

## Cross-Validation

Para evaluar mejor la capacidad de nuestro modelo para predecir, se puede repetir el proceso de comparar entre nuestra predicción y la *test dataset* más sistemáticamente, para otras "*test datasets*" dentro de nuestro periodo observado.  
  
La librería ```statsforecast```  tiene una función para ello: ```cross_validation()```.

En primer lugar, creamos el objeto de *statsforecast* con el modelo o modelos que deseamos usar:

In [ ]:
modelo = StatsForecast(models=[AutoARIMA(season_length=12,
                                         max_p=2, max_d=1, max_q=2, max_P=1, max_D=1, max_Q=1,
                                          )],freq='MS')

Entonces, podemos usar el método `cross_validation()`, que acepta los siguientes argumentos:

* `df`: train dataframe con formato `StatsForecast`.
* `h` (int): representa los `h` periodos en el futuro que se predicirán.
* `step_size` (int): número de periodos entre cada ventana.
* `n_windows` (int): número de ventanas utilizadas para la validación cruzada, es decir, el número de predicciones que se evaluarán.

Como ejemplo, usaremos 3 ventanas de 12 meses:



In [ ]:
cv_df = modelo.cross_validation(
    df = medic,
    h = 24,
    step_size = 24,
    n_windows = 3, fitted=True
  )

El objeto **cv_df** es una dataframe que incluye las siguientes columnas:

* `unique_id`: identificador de la serie.

* `ds`: periodo.

* `cutoff`: el último periodo para las `n_windows`.

* `y`: valor observado de la serie.

* `model`: columnas con el nombre del modelo y el valor predicho.

In [ ]:
cv_df.head()

Para acceder a las *insample cross validated predictions*, en caso de que lo necesitemos para algo, procedemos así:
``` modelo.cross_validation_fitted_values() ```

Dibujamos la *cross validation* realizada:

In [ ]:
import matplotlib.dates as mdates
cv_df.set_index('ds', drop=False, inplace=True)
cutoff=cv_df.cutoff.unique()


fig,ax=plt.subplots(1,3,dpi=100)
for i in range(3):
  medic_temp=medic.loc[medic.ds<cutoff[i],"y"]
  ax[i].plot(medic_temp.iloc[-24:], color="b")
  ax[i].plot(cv_df.loc[cv_df.cutoff==cutoff[i],"y"], color="k")
  ax[i].plot(cv_df.loc[cv_df.cutoff==cutoff[i],"AutoARIMA"], color="r")
  ax[i].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1,7)))
  ax[i].xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax[i].xaxis.get_major_locator()))
fig.suptitle("Cross validation: predicción de ventas de medicamentos")
plt.show()

Finalmente, calculamos las métricas sobre exactitud del modelo para predecir (notad, que ahora se basa no solo en 24 observaciones (la test dataset original) sino en las tres ventanas de validación (24*3)).

In [ ]:
forecast_accuracy(cv_df['AutoARIMA'], cv_df.y)

## AutoArima con variable exógena:

`statsforecast` también permite incorporar **regresores exógenos** al modelo AutoARIMA.  

Cuando se usan regresores exógenos, es **fundamental** asegurarnos que los periodos en cada tabla son los correctos.

Aplicado a nuestro ejemplo:

|Variable endógena (y) | variable exógena (x) |
|----------------------|----------------------|
| **División entre train y test dataset:** |   |
| Training dataset: 1991/07 - 2006/06     | X_in: 1991/07 - 2006/06        |
| Test dataset: 2006/07 - 2008/06         | X_out: 2006/07 - 2008/06        |
| **Predicción hacia el futuro:** |   |
| Datos completos: 1991/07 - 2008/06 | X_in: 1991/07 - 2008/06
| Predicción hacia el futuro 6 periodos |  X_out: 2008/07-2008/12 |

*Nótese que solo podemos predecir hacia el futuro 6 periodos porque nuestra variable exógena solo está disponible para dichos 6 periodos, no más.*

In [ ]:
visit = pd.read_excel("https://github.com/otoperalias/Coyuntura/raw/refs/heads/main/clases/datos/medic_visitas_doctor.xlsx", index_col=0, parse_dates=True)
visit["unique_id"]="ventas_medic"
visit["ds"]=visit.index
visit.head()

Dividimos la variable exógena en train/test:

In [ ]:
X_in=visit.loc[train.index]
X_out=visit.loc[test.index]

En `statsforecast`, a diferencia de `statsmodels`, hay que incluir en la misma dataframe tanto la variable endógena (y) como la variable exógena. Para ello, usamos la función `merge()`:

In [ ]:
train = train.merge(X_in, how = 'left', on = ['ds','unique_id'])
train.head()

In [ ]:
modelo = StatsForecast(models=[AutoARIMA(season_length=12,
                                         max_p=2, max_d=1, max_q=2, max_P=1, max_D=1, max_Q=1,    # Para replicar los valores máximos establecidos en el método manual
                                          )],freq='MS')

A la hora de hacer la predicción, incluimos la variable exógena correspondiente a la test dataset (X_out):

In [ ]:
fcst = modelo.forecast(df=train, h=24, X_df=X_out, level=[95])
fcst.head()

In [ ]:
# Gráfico
fcst.set_index(fcst.ds, drop=False, inplace=True)
train.set_index(train.ds, drop=False, inplace=True)

fig,ax=plt.subplots(figsize=(12,8))
ax.plot(train[-60:]['y'], label="Training period", color="b") #con [-60:] evito representar la serie completa
ax.plot(test['y'], label="Actual values", color="k")
ax.plot(fcst['AutoARIMA'], label="Predicted", color='r')
ax.fill_between(fcst.index,
                 fcst['AutoARIMA-lo-95'],
                 fcst['AutoARIMA-hi-95'],
                 color='k', alpha=.15)
ax.legend()
ax.set_title("SARIMAX - Predicción de ventas de medicamentos con X")
plt.show()



In [ ]:
forecast_accuracy(fcst['AutoARIMA'], test.y)

Podemos comparar estas métricas con las del modelo autovariante, es decir, sin variable exógena:

In [ ]:
# Predicción con modelo univariante:
fcst_u = modelo.forecast(df=train[['unique_id', 'ds', 'y']], h=24)
fcst_u.set_index(fcst_u.ds, drop=False, inplace=True)

forecast_accuracy(fcst_u['AutoARIMA'], test['y'])

Vemos que el nuevo modelo es mucho más preciso a la hora de predecir.

Además, podemos hacer *cross validation*, ya que la función `cross_validation()` también permite variables exógenas:

In [ ]:
# Recuérdese que para hacer cross validation, usamos la dataset completa:

medic=medic.merge(visit, on=['ds','unique_id'], how='left')
# al hacer el merge con el argumento how="left" descarto las observaciones que están en la tabla visit pero no en la tabla medic.

cv_df = modelo.cross_validation(
    df = medic,
    h = 24,
    step_size = 24,
    n_windows = 3, fitted=True
  )

In [ ]:
forecast_accuracy(cv_df['AutoARIMA'], cv_df.y)

Finalmente, predecimos hacia el futuro:

In [ ]:
X_out=visit[-6:]
fcst = modelo.forecast(df=medic, h=6, X_df=X_out, level=[80,90,95,99])

# Gráfico
fcst.set_index(fcst.ds, drop=False, inplace=True)

fig,ax=plt.subplots(figsize=(10,7))
ax.plot(medic.loc["2006":,'y'], label="Serie observada", color="b") #con [-60:] evito representar la serie completa
ax.plot(fcst['AutoARIMA'], label="Predicted", color='r')
for ic in ["80","90","95","99"]:
  ax.fill_between(fcst.index,
                  fcst['AutoARIMA-lo-'+ic],
                  fcst['AutoARIMA-hi-'+ic],
                  color='k', alpha=0.05)
ax.legend()
ax.set_title("SARIMAX - Predicción de ventas de medicamentos con X\n", size=16)
plt.show()
